In [ ]:
#@title **🌏 Cài đặt**
import os

from ipywidgets import Button
from IPython.display import clear_output

#@markdown **Cài đặt ở chế độ không in ra nhật ký**
cai_dat_trong_im_lang = True #@param {type:"boolean"}

print("👩🏻‍💻 Cài đặt...")

if cai_dat_trong_im_lang:
    !git clone https://github.com/PhamHuynhAnh16/Vietnamese-RVC /kaggle/working/Vietnamese_RVC > /dev/null 2>&1
    !apt-get -y install libportaudio2 -qq > /dev/null 2>&1

    !pip install uv > /dev/null 2>&1
    !uv pip install -r /kaggle/working/Vietnamese_RVC/requirements.txt --no-cache-dir -q > /dev/null 2>&1
    !uv pip install -q ngrok --no-cache-dir > /dev/null 2>&1
    !npm install -g -q localtunnel > /dev/null 2>&1

    clear_output()
else:
    !git clone https://github.com/PhamHuynhAnh16/Vietnamese-RVC /kaggle/working/Vietnamese_RVC
    !apt-get -y install libportaudio2

    !pip install uv
    !uv pip install -r /kaggle/working/Vietnamese_RVC/requirements.txt --no-cache-dir
    !uv pip install -q ngrok --no-cache-dir
    !npm install -g -q localtunnel

#@markdown **💻 Cài đặt sẽ mất khoảng 2 phút để hoàn tất!**
Button(description="\u2714 Hoàn Tất!", button_style="success")

In [ ]:
#@title **📱 Mở giao diện sử dụng**
import os
import json

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
configs_json = os.path.join("main", "configs", "config.json")

def change_language(lang):
    configs = json.load(open(configs_json, "r"))

    if lang != configs["language"]:
        configs["language"] = lang

        with open(configs_json, "w") as f:
            json.dump(configs, f, indent=4)

%cd /kaggle/working/Vietnamese_RVC
#@markdown **Phương thức trình bày giao diện webui**
phuong_thuc = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]
#@markdown **Để trải nghiệm hết tính năng hãy dùng giao diện:) Còn muốn đơn giản thì không dùng giao diện**
ngon_ngu = "vi-VN" #@param ["vi-VN", "en-US"]
#@markdown **Nếu biết có thể sử dụng biểu đồ để kiểm tra huấn luyện quá sức 👍**
su_dung_bieu_do = False #@param {type:"boolean"}

if su_dung_bieu_do:
    %load_ext tensorboard
    %tensorboard --logdir ./assets/logs --port=6870

change_language(ngon_ngu)

if not os.path.exists("/kaggle/working/Vietnamese_RVC/assets/models/predictors/rmvpe.pt"):
    !wget https://huggingface.co/AnhP/Vietnamese-RVC-Project/resolve/main/predictors/rmvpe.pt -O /kaggle/working/Vietnamese_RVC/assets/models/predictors/rmvpe.pt > /dev/null 2>&1

if not os.path.exists("/kaggle/working/Vietnamese_RVC/assets/models/embedders/hubert_base.pt"):
    !wget https://huggingface.co/AnhP/Vietnamese-RVC-Project/resolve/main/embedders/fairseq/hubert_base.pt -O /kaggle/working/Vietnamese_RVC/assets/models/embedders/hubert_base.pt > /dev/null 2>&1


match phuong_thuc:
  case 'gradio':
    !python main/app/app.py --listen --share --client
  case 'localtunnel':
    !echo Mật khẩu IP của bạn: $(curl --silent https://ipv4.icanhazip.com)
    !lt --port 7860 & python main/app/app.py --listen --client
  case 'ngrok':
    import ngrok

    ngrok.kill()

    print("Nếu bạn đã chọn phương thức 'ngrok', hãy lấy mã xác thực của bạn tại đây: https://dashboard.ngrok.com/get-started/your-authtoken")
    ngrok_token = input("Nhập mã xác thực: ")

    listener = await ngrok.forward(7860, authtoken=ngrok_token)
    print(f"Liên kết Ngrok: {listener.url()}")

    !python main/app/app.py --listen --client